In [5]:
!pip install pandas numpy matplotlib scipy PyWavelets scikit-learn joblib lightgbm openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 37.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.fft import fft
import pywt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from scipy.stats import entropy
import zipfile
import tempfile
from collections import defaultdict
import json
import lightgbm as lgb
import re
import warnings

# Tắt các cảnh báo không cần thiết
warnings.filterwarnings("ignore", category=UserWarning)  # Tắt cảnh báo wavelet
warnings.filterwarnings("ignore", category=FutureWarning)  # Tắt cảnh báo tương lai từ sklearn/seaborn

# Tạo thư mục output ngay đầu mã
output_dir = 'model_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Bước 1: Giải nén file ZIP và đọc dữ liệu
zip_file = 'Wind Turbine Blades Fault Diagnosis based on Vibration Dataset Analysis.zip'

# File cấu hình ánh xạ tên file với trạng thái và tốc độ gió
config_file = 'file_config.json'
file_config = {
    'H-for-Vw=5.0.csv': {'state': 'healthy', 'wind_speed': 5.0},
    'Crack Fault-Vw=5.csv': {'state': 'crack', 'wind_speed': 5.0},
    'Erosion Fault state-Vw=5.csv': {'state': 'erosion', 'wind_speed': 5.0},
    'unbalance fault state-Vw=5.csv': {'state': 'unbalance', 'wind_speed': 5.0},
    'twist fault when Vw=5.xlsx': {'state': 'twist', 'wind_speed': 5.0},
    'H-for Vw=1.3.csv': {'state': 'healthy', 'wind_speed': 1.3},
    'Crack State-Vw=1.3.csv': {'state': 'crack', 'wind_speed': 1.3},
    'Erosion fault state-Vw=1.3.csv': {'state': 'erosion', 'wind_speed': 1.3},
    'UnbalanceState-Vw=1.3.csv': {'state': 'unbalance', 'wind_speed': 1.3},
    'twist fault when Vw=1.3.xlsx': {'state': 'twist', 'wind_speed': 1.3},
    'H for Vw=2.3.csv': {'state': 'healthy', 'wind_speed': 2.3},
    'unbalance fault state=Vw=2.3.csv': {'state': 'unbalance', 'wind_speed': 2.3},
    'H-for-Vw=3.2.csv': {'state': 'healthy', 'wind_speed': 3.2},
    'twist faultwhenVw=3.2.xlsx': {'state': 'twist', 'wind_speed': 3.2},
    'H-for-Vw=3.7.csv': {'state': 'healthy', 'wind_speed': 3.7},
    'H-for-Vw=4.5.csv': {'state': 'healthy', 'wind_speed': 4.5},
    'crack fault state-Vw=4.5.csv': {'state': 'crack', 'wind_speed': 4.5},
    'H-Vw=5.3.csv': {'state': 'healthy', 'wind_speed': 5.3},
    'Crack Fault-Vw=5.4.csv': {'state': 'crack', 'wind_speed': 5.4},
    'Erosion fault state-Vw=5.3.csv': {'state': 'erosion', 'wind_speed': 5.3},
    'unbalance fault state-Vw=4.7.csv': {'state': 'unbalance', 'wind_speed': 4.7},
    'unbalance fault state-Vw=4.2.csv': {'state': 'unbalance', 'wind_speed': 4.2},
    'Crack state-Vw=2.8.csv': {'state': 'crack', 'wind_speed': 2.8},
    'Erosion fault state-Vw=2.1.csv': {'state': 'erosion', 'wind_speed': 2.1},
    'Erosion fault state-Vw=2.8.csv': {'state': 'erosion', 'wind_speed': 2.8},
    'unbalnce fault state-Vw=3.csv': {'state': 'unbalance', 'wind_speed': 3.0},
    'Erosion Fault State-Vw=4.2.csv': {'state': 'erosion', 'wind_speed': 4.2},
    'unbalance fault state-Vw=3.4.csv': {'state': 'unbalance', 'wind_speed': 3.4},
    'twist fault when Vw=4.7.xlsx': {'state': 'twist', 'wind_speed': 4.7},
    'crack fault-Vw=4.csv': {'state': 'crack', 'wind_speed': 4.0},
    'Crack state-Vw=3.3.csv': {'state': 'crack', 'wind_speed': 3.3},
    'Erosion fault state-Vw=3.4.csv': {'state': 'erosion', 'wind_speed': 3.4},
    'twist fault when Vwind=4.xlsx': {'state': 'twist', 'wind_speed': 4.0},
    'twsist faut when Vw=2.xlsx': {'state': 'twist', 'wind_speed': 2.0},
    'twist fault when Vw=5.3.xlsx': {'state': 'twist', 'wind_speed': 5.3},
    'H-for-Vw=5.csv': {'state': 'healthy', 'wind_speed': 5.0}  # Thêm file bị thiếu
}

# Lưu file cấu hình nếu chưa tồn tại
if not os.path.exists(config_file):
    with open(config_file, 'w') as f:
        json.dump(file_config, f, indent=4)

with tempfile.TemporaryDirectory() as temp_dir:
    print(f"Giải nén file ZIP vào thư mục tạm: {temp_dir}")
    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            print("Nội dung file ZIP:")
            for file in zip_ref.namelist():
                print(file)
            zip_ref.extractall(temp_dir)
    except zipfile.BadZipFile:
        raise ValueError(f"Lỗi: {zip_file} không phải là file ZIP hợp lệ hoặc bị hỏng")

    def find_data_files(directory):
        data_files = []
        for root, _, files in os.walk(directory):
            for file in files:
                if file.endswith('.csv') or file.endswith('.xlsx'):
                    data_files.append(os.path.join(root, file))
        return data_files

    data_files = find_data_files(temp_dir)
    if not data_files:
        raise FileNotFoundError("Không tìm thấy file CSV hoặc XLSX trong file ZIP đã giải nén")

    # Đọc file cấu hình
    try:
        with open(config_file, 'r') as f:
            file_config = json.load(f)
    except FileNotFoundError:
        print(f"Không tìm thấy {config_file}. Sử dụng cấu hình mặc định.")

    dataset = []
    for data_file in data_files:
        file = os.path.basename(data_file)
        print(f"Xử lý file: {file}")
        try:
            if file.endswith('.csv'):
                df = pd.read_csv(data_file, sep=';|,', engine='python')
            elif file.endswith('.xlsx'):
                df = pd.read_excel(data_file)
        except Exception as e:
            print(f"Lỗi khi đọc file {file}: {e}")
            continue

        print(f"Các cột trong {file}: {df.columns}")

        # Lấy trạng thái và tốc độ gió từ file cấu hình
        config_entry = file_config.get(file, None)
        if config_entry is None:
            print(f"File {file} không có trong file cấu hình. Bỏ qua.")
            continue

        state = config_entry['state']
        wind_speed = config_entry['wind_speed']

        # Chuẩn hóa tên cột
        column_map = {
            'Time - Voltage_1;Amplitude - Voltage_1': 'amplitude',
            'Time - sec;Amplitude - g': 'amplitude',
            'Amplitude - Voltage_1': 'amplitude',
            'Time - Voltage_1': 'time',
            'Time - sec': 'time'
        }
        df = df.rename(columns=column_map)

        if 'amplitude' not in df.columns:
            for col in df.columns:
                if 'Amplitude' in col:
                    df['amplitude'] = df[col]
                elif ';' in str(df[col].iloc[0]):
                    df['amplitude'] = df[col].apply(lambda x: float(x.split(';')[-1]) if isinstance(x, str) else float(x))
                    df['time'] = df[col].apply(lambda x: float(x.split(';')[0]) if isinstance(x, str) else np.nan)
                    break

        if 'amplitude' not in df.columns:
            print(f"Không tìm thấy cột amplitude trong {file}")
            continue

        if 'time' not in df.columns:
            df['time'] = np.arange(len(df)) / 1000.0

        df['state'] = state
        df['wind_speed'] = wind_speed
        dataset.append(df)

if not dataset:
    raise ValueError("Không có file nào được xử lý thành công. Kiểm tra file cấu hình và dữ liệu.")

full_data = pd.concat(dataset, ignore_index=True)

# Bước 2: Tiền xử lý dữ liệu
print("Các cột trong full_data:", full_data.columns)
print("Giá trị thiếu:\n", full_data.isna().sum())
print("Kiểm tra giá trị bất thường trong amplitude:")
print(full_data['amplitude'].describe())
print("Số giá trị NaN hoặc Inf:", full_data['amplitude'].isna().sum() + np.isinf(full_data['amplitude']).sum())
full_data = full_data.dropna(subset=['amplitude'])  # Loại bỏ các mẫu có amplitude NaN
scaler = StandardScaler()
full_data['amplitude_scaled'] = scaler.fit_transform(full_data[['amplitude']])

# Bước 3: Trích xuất đặc trưng
def extract_features(signal, fs=1000):
    if isinstance(signal, pd.Series):
        signal = signal.to_numpy()
    elif not isinstance(signal, np.ndarray):
        signal = np.array(signal)

    features = {
        'mean': signal.mean(),
        'std': signal.std(),
        'peak': np.abs(signal).max(),
        'rms': ((signal ** 2).mean()) ** 0.5,
        'skewness': pd.Series(signal).skew(),
        'kurtosis': pd.Series(signal).kurt(),
        'crest_factor': np.abs(signal).max() / (((signal ** 2).mean()) ** 0.5) if ((signal ** 2).mean()) > 0 else 0,
        'signal_entropy': entropy(np.histogram(signal, bins=10, density=True)[0]) if np.sum(signal) > 0 else 0,
        'zero_crossing_rate': ((signal[:-1] * signal[1:] < 0).sum()) / len(signal)
    }
    fft_vals = np.abs(fft(signal))[:len(signal) // 2]
    freqs = np.linspace(0, fs / 2, len(fft_vals))
    low_freq = fft_vals[(freqs <= 50)]
    mid_freq = fft_vals[(freqs > 50) & (freqs <= 200)]
    high_freq = fft_vals[(freqs > 200) & (freqs <= 500)]
    features['low_freq_energy'] = np.log1p(np.sum(low_freq ** 2)) if len(low_freq) > 0 else 0
    features['mid_freq_energy'] = np.log1p(np.sum(mid_freq ** 2)) if len(mid_freq) > 0 else 0
    features['high_freq_energy'] = np.log1p(np.sum(high_freq ** 2)) if len(low_freq) > 0 else 0
    coeffs = pywt.wavedec(signal, 'db4', level=2)  # Sử dụng db4 và level=2
    features['wavelet_energy'] = sum(np.sum(c ** 2) for c in coeffs)
    features['wavelet_std'] = np.std(coeffs[-1])
    features['approx_energy'] = np.sum(coeffs[0] ** 2)
    features['detail_energy_1'] = np.sum(coeffs[1] ** 2)
    features['detail_energy_2'] = np.sum(coeffs[2] ** 2)
    return features

def augment_signal(signal, noise_factor=0.02, shift_max=5, scale_factor_range=(0.9, 1.1)):
    noise = np.random.normal(0, noise_factor, len(signal))
    noisy_signal = signal + noise
    shift = np.random.randint(-shift_max, shift_max)
    shifted_signal = np.roll(signal, shift)
    scale_factor = np.random.uniform(scale_factor_range[0], scale_factor_range[1])
    scaled_signal = signal * scale_factor
    return noisy_signal, shifted_signal, scaled_signal

# Tự động xác định segment_length
def determine_segment_length(group_sizes):
    min_samples = min(group_sizes)
    if min_samples < 10:
        raise ValueError("Nhóm nhỏ nhất có ít hơn 10 mẫu, không đủ để trích xuất đặc trưng.")
    elif min_samples < 100:
        return 50
    elif min_samples < 200:
        return 100
    else:
        return 200

features_list = []
group_sizes = [len(group) for _, group in full_data.groupby(['state', 'wind_speed'])]
segment_length = determine_segment_length(group_sizes)
print(f"Đã chọn segment_length={segment_length} dựa trên kích thước nhóm.")

for (state, wind_speed), group in full_data.groupby(['state', 'wind_speed']):
    print(f"Nhóm {state}, wind_speed={wind_speed}: {len(group)} mẫu")
    for i in range(0, len(group), segment_length):
        segment = group['amplitude'].iloc[i:i + segment_length]
        if len(segment) >= 10:
            if len(segment) < segment_length:
                segment = np.pad(segment, (0, segment_length - len(segment)), mode='constant')
            features = extract_features(segment)
            features['state'] = state
            features['wind_speed'] = wind_speed
            features_list.append(features)
            noisy_segment, shifted_signal, scaled_segment = augment_signal(segment, noise_factor=0.02, shift_max=5)
            for aug_segment in [noisy_segment, shifted_signal, scaled_segment]:
                features = extract_features(aug_segment)
                features['state'] = state
                features['wind_speed'] = wind_speed
                features_list.append(features)
        else:
            print(
                f"Bỏ qua đoạn trong nhóm {state}, wind_speed={wind_speed}: chỉ có {len(segment)} mẫu, cần ít nhất 10")

features_df = pd.DataFrame(features_list)
print("Số mẫu trong features_df:", len(features_df))
if features_df.empty:
    raise ValueError("Không tạo được đặc trưng nào. Kiểm tra segment_length và số mẫu mỗi nhóm.")
print("Phân bố lớp:\n", features_df['state'].value_counts())

# Biểu đồ khái quát: Phân bố lớp
plt.figure(figsize=(8, 6))
features_df['state'].value_counts().plot(kind='bar', color='skyblue')
plt.xlabel('Trạng thái')
plt.ylabel('Số lượng mẫu')
plt.title('Phân bố lớp trong tập dữ liệu')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'class_distribution.png'))
plt.show()

# Biểu đồ khái quát: Boxplot cho các đặc trưng quan trọng
key_features = ['mean', 'std', 'peak', 'wavelet_energy']
plt.figure(figsize=(12, 8))
for i, feature in enumerate(key_features, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(x='state', y=feature, data=features_df)
    plt.title(f'Phân bố {feature} theo trạng thái')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'feature_boxplots.png'))
plt.show()

# Ma trận tương quan trước PCA
plt.figure(figsize=(12, 10))
corr_matrix = features_df.drop(['state', 'wind_speed'], axis=1).corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Ma trận tương quan của các đặc trưng trước PCA')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'correlation_matrix_pre_pca.png'))
plt.show()

# Bước 4: Áp dụng PCA để giảm chiều
X = features_df.drop(['state', 'wind_speed'], axis=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_components = min(8, X_scaled.shape[1])  # Giảm xuống 8 thành phần
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)
print(f"Đã giảm chiều từ {X_scaled.shape[1]} xuống {X_pca.shape[1]} đặc trưng với PCA.")
print(f"Tỷ lệ phương sai được giải thích: {sum(pca.explained_variance_ratio_):.4f}")

pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
features_df_pca = pd.DataFrame(X_pca, columns=pca_columns)
features_df_pca['state'] = features_df['state']
features_df_pca['wind_speed'] = features_df['wind_speed']

# Ma trận tương quan sau PCA
plt.figure(figsize=(8, 6))
corr_matrix_pca = features_df_pca.drop(['state', 'wind_speed'], axis=1).corr()
sns.heatmap(corr_matrix_pca, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Ma trận tương quan của các thành phần PCA')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'correlation_matrix_post_pca.png'))
plt.show()

# Biểu đồ khái quát: Phân tán PCA (PC1 vs PC2)
plt.figure(figsize=(10, 8))
for state in features_df_pca['state'].unique():
    subset = features_df_pca[features_df_pca['state'] == state]
    plt.scatter(subset['PC1'], subset['PC2'], label=state, alpha=0.6)
plt.xlabel('Thành phần chính 1 (PC1)')
plt.ylabel('Thành phần chính 2 (PC2)')
plt.title('Phân tán PCA của dữ liệu theo trạng thái')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'pca_scatter.png'))
plt.show()

# Bước 5: Trực quan hóa tín hiệu dao động (cho nhiều tốc độ gió)
wind_speeds = [1.3, 5.0]
for ws in wind_speeds:
    plt.figure(figsize=(10, 6))
    for state, group in full_data[full_data['wind_speed'] == ws].groupby('state'):
        plt.plot(group['time'][:500], group['amplitude'][:500], label=state)
    plt.xlabel('Thời gian (s)')
    plt.ylabel('Biên độ (g)')
    plt.title(f'Tín hiệu dao động tại tốc độ gió {ws} m/s')
    plt.legend()
    plt.savefig(os.path.join(output_dir, f'vibration_signals_ws_{ws}.png'))
    plt.show()

# Bước 6: Hàm đánh giá tất cả mô hình
def evaluate_all_models(X, y, model_configs, cv=5, test_size=0.15, random_state=42):
    results = defaultdict(dict)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    le = LabelEncoder()
    y_train_encoded = le.fit_transform(y_train)
    y_test_encoded = le.transform(y_test)

    for model_name, config in model_configs.items():
        print(f"\nĐang huấn luyện mô hình: {model_name}")
        model = config['model']
        param_grid = config['param_grid']

        grid_search = GridSearchCV(
            model, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=1
        )
        grid_search.fit(X_train, y_train_encoded)

        results[model_name]['best_score'] = grid_search.best_score_
        results[model_name]['best_params'] = grid_search.best_params_
        results[model_name]['best_model'] = grid_search.best_estimator_

        y_pred = grid_search.best_estimator_.predict(X_test)
        report = classification_report(
            y_test_encoded, y_pred, target_names=le.classes_, zero_division=0, output_dict=True
        )
        results[model_name]['classification_report'] = report

        # Ma trận nhầm lẫn
        cm = confusion_matrix(y_test_encoded, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
        plt.title(f'Ma trận nhầm lẫn - {model_name}')
        plt.xlabel('Dự đoán')
        plt.ylabel('Thực tế')
        plt.savefig(os.path.join(output_dir, f'confusion_matrix_{model_name.lower()}.png'))
        plt.show()

        if hasattr(grid_search.best_estimator_, 'feature_importances_'):
            feature_importance = pd.DataFrame({
                'feature': X.columns,
                'importance': grid_search.best_estimator_.feature_importances_
            }).sort_values(by='importance', ascending=False)
            results[model_name]['feature_importance'] = feature_importance
        else:
            results[model_name]['feature_importance'] = None

        print(f"Điểm kiểm tra chéo tốt nhất ({model_name}): {grid_search.best_score_:.4f}")
        print(f"Tham số tốt nhất ({model_name}): {grid_search.best_params_}")
        print(f"Báo cáo phân loại ({model_name}):\n", classification_report(
            y_test_encoded, y_pred, target_names=le.classes_, zero_division=0
        ))
        if results[model_name]['feature_importance'] is not None:
            print(f"Tầm quan trọng đặc trưng ({model_name}):\n", results[model_name]['feature_importance'])
        else:
            print(f"Không thể tính tầm quan trọng đặc trưng cho {model_name}")

        joblib.dump(grid_search.best_estimator_, os.path.join(output_dir, f'wind_turbine_fault_model_{model_name.lower()}.pkl'))
        print(f"Mô hình {model_name} đã được lưu dưới dạng 'wind_turbine_fault_model_{model_name.lower()}.pkl'")

    return results, le, X_test, y_test

# Cấu hình các mô hình
model_configs = {
    'LightGBM': {
        'model': lgb.LGBMClassifier(
            random_state=42,
            class_weight='balanced',
            force_col_wise=True,
            verbose=-1,
            min_child_samples=5,
            min_split_gain=0.0
        ),
        'param_grid': {
            'n_estimators': [100, 200, 300],
            'max_depth': [5, 7, 10],
            'learning_rate': [0.01, 0.1],
            'num_leaves': [15, 31, 50],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0]
        }
    },
    'MLP': {
        'model': MLPClassifier(random_state=42, max_iter=1000),
        'param_grid': {
            'hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'activation': ['relu', 'tanh'],
            'learning_rate_init': [0.001, 0.01],
            'alpha': [0.0001, 0.001]
        }
    },
    'SVM': {
        'model': SVC(random_state=42, class_weight='balanced'),
        'param_grid': {
            'C': [0.01, 0.1, 1, 10, 100],
            'kernel': ['rbf', 'linear', 'poly'],
            'gamma': [0.01, 0.1, 'scale'],
            'degree': [2, 3]
        }
    }
}

# Bước 7: Đánh giá tất cả mô hình với dữ liệu PCA
X_pca = features_df_pca.drop(['state', 'wind_speed'], axis=1)
y = features_df_pca['state']
results, label_encoder, X_test, y_test = evaluate_all_models(X_pca, y, model_configs)

# Bước 8: So sánh hiệu suất
print("\nSo sánh hiệu suất các mô hình:")
comparison = pd.DataFrame({
    'Model': [],
    'Cross-Validation Accuracy': [],
    'Test Accuracy': [],
    'Macro F1-Score': []
})
for model_name, result in results.items():
    comparison = pd.concat([comparison, pd.DataFrame({
        'Model': [model_name],
        'Cross-Validation Accuracy': [result['best_score']],
        'Test Accuracy': [result['classification_report']['accuracy']],
        'Macro F1-Score': [result['classification_report']['macro avg']['f1-score']]
    })], ignore_index=True)

comparison = comparison.sort_values(by='Test Accuracy', ascending=False)
print(comparison)
comparison.to_csv(os.path.join(output_dir, 'model_comparison.csv'), index=False)

# Bước 9: Vẽ biểu đồ Cross-Validation Accuracy
plt.figure(figsize=(8, 6))
plt.bar(comparison['Model'], comparison['Cross-Validation Accuracy'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.xlabel('Mô hình')
plt.ylabel('Cross-Validation Accuracy')
plt.title('So sánh Cross-Validation Accuracy của các mô hình')
plt.ylim(0, 1)
for i, v in enumerate(comparison['Cross-Validation Accuracy']):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'cross_validation_accuracy.png'))
plt.show()

ModuleNotFoundError: No module named 'pywt'

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.fft import fft
import pywt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import joblib
from scipy.stats import entropy
import os
from collections import Counter
import warnings

# Tắt các cảnh báo không cần thiết
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Thư mục chứa mô hình và tham số đã lưu
output_dir = 'model_outputs'

# Hàm trích xuất đặc trưng (giữ nguyên từ mã gốc)
def extract_features(signal, fs=1000):
    if isinstance(signal, pd.Series):
        signal = signal.to_numpy()
    elif not isinstance(signal, np.ndarray):
        signal = np.array(signal)

    features = {
        'mean': signal.mean(),
        'std': signal.std(),
        'peak': np.abs(signal).max(),
        'rms': ((signal ** 2).mean()) ** 0.5,
        'skewness': pd.Series(signal).skew(),
        'kurtosis': pd.Series(signal).kurt(),
        'crest_factor': np.abs(signal).max() / (((signal ** 2).mean()) ** 0.5) if ((signal ** 2).mean()) > 0 else 0,
        'signal_entropy': entropy(np.histogram(signal, bins=10, density=True)[0]) if np.sum(signal) > 0 else 0,
        'zero_crossing_rate': ((signal[:-1] * signal[1:] < 0).sum()) / len(signal)
    }
    fft_vals = np.abs(fft(signal))[:len(signal) // 2]
    freqs = np.linspace(0, fs / 2, len(fft_vals))
    low_freq = fft_vals[(freqs <= 50)]
    mid_freq = fft_vals[(freqs > 50) & (freqs <= 200)]
    high_freq = fft_vals[(freqs > 200) & (freqs <= 500)]
    features['low_freq_energy'] = np.log1p(np.sum(low_freq ** 2)) if len(low_freq) > 0 else 0
    features['mid_freq_energy'] = np.log1p(np.sum(mid_freq ** 2)) if len(mid_freq) > 0 else 0
    features['high_freq_energy'] = np.log1p(np.sum(high_freq ** 2)) if len(low_freq) > 0 else 0
    coeffs = pywt.wavedec(signal, 'db4', level=2)
    features['wavelet_energy'] = sum(np.sum(c ** 2) for c in coeffs)
    features['wavelet_std'] = np.std(coeffs[-1])
    features['approx_energy'] = np.sum(coeffs[0] ** 2)
    features['detail_energy_1'] = np.sum(coeffs[1] ** 2)
    features['detail_energy_2'] = np.sum(coeffs[2] ** 2)
    return features

# Hàm xử lý và dự đoán
def predict_fault(signal=None, csv_file=None, segment_length=200):
    # Kiểm tra input
    if signal is None and csv_file is None:
        # Tạo tín hiệu ngẫu nhiên nếu không có input
        signal = np.random.normal(0, 1, segment_length)
        print("Không có tín hiệu hoặc file CSV được cung cấp. Tạo tín hiệu ngẫu nhiên.")

    # Tải các tham số và mô hình đã lưu
    try:
        scaler = joblib.load(os.path.join(output_dir, 'scaler.pkl'))
        pca = joblib.load(os.path.join(output_dir, 'pca.pkl'))
        label_encoder = joblib.load(os.path.join(output_dir, 'label_encoder.pkl'))
        models = {
            'LightGBM': joblib.load(os.path.join(output_dir, 'wind_turbine_fault_model_lightgbm.pkl')),
            'MLP': joblib.load(os.path.join(output_dir, 'wind_turbine_fault_model_mlp.pkl')),
            'SVM': joblib.load(os.path.join(output_dir, 'wind_turbine_fault_model_svm.pkl'))
        }
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Không tìm thấy file mô hình hoặc tham số: {e}. Vui lòng chạy mã huấn luyện trước.")

    # Xử lý dữ liệu đầu vào
    if csv_file:
        try:
            df = pd.read_csv(csv_file, sep=';|,', engine='python')
            if 'amplitude' not in df.columns:
                for col in df.columns:
                    if 'Amplitude' in col or 'amplitude' in col.lower():
                        df['amplitude'] = df[col]
                        break
                    elif ';' in str(df[col].iloc[0]):
                        df['amplitude'] = df[col].apply(lambda x: float(x.split(';')[-1]) if isinstance(x, str) else float(x))
                        break
            if 'amplitude' not in df.columns:
                raise ValueError("File CSV không chứa cột 'amplitude'.")
            signal = df['amplitude'].to_numpy()
        except Exception as e:
            raise ValueError(f"Lỗi khi đọc file CSV: {e}")

    # Chuyển tín hiệu thành numpy array
    signal = np.array(signal, dtype=float)
    if len(signal) < 10:
        raise ValueError("Tín hiệu quá ngắn, cần ít nhất 10 mẫu.")

    # Chia tín hiệu thành các đoạn
    segments = []
    if len(signal) < segment_length:
        # Đệm tín hiệu ngắn bằng 0
        signal = np.pad(signal, (0, segment_length - len(signal)), mode='constant')
        segments.append(signal)
    else:
        for i in range(0, len(signal), segment_length):
            segment = signal[i:i + segment_length]
            if len(segment) >= 10:
                if len(segment) < segment_length:
                    segment = np.pad(segment, (0, segment_length - len(segment)), mode='constant')
                segments.append(segment)

    if not segments:
        raise ValueError("Không tạo được đoạn tín hiệu nào hợp lệ.")

    # Trích xuất đặc trưng
    features_list = []
    for segment in segments:
        features = extract_features(segment)
        features_list.append(features)

    features_df = pd.DataFrame(features_list)

    # Chuẩn hóa và giảm chiều
    X_scaled = scaler.transform(features_df)
    X_pca = pca.transform(X_scaled)

    # Dự đoán
    predictions = {}
    all_labels = []
    for model_name, model in models.items():
        y_pred = model.predict(X_pca)
        y_pred_labels = label_encoder.inverse_transform(y_pred)
        all_labels.extend(y_pred_labels)

        try:
            y_proba = model.predict_proba(X_pca)
            proba_dict = {label: proba for label, proba in zip(label_encoder.classes_, y_proba[0])}
        except AttributeError:
            proba_dict = None

        predictions[model_name] = {
            'labels': y_pred_labels,
            'probabilities': proba_dict
        }

    # Kết hợp dự đoán bằng voting
    vote_counts = Counter(all_labels)
    ensemble_label = vote_counts.most_common(1)[0][0]
    predictions['Ensemble'] = {'labels': [ensemble_label], 'probabilities': None}

    return predictions

# Hàm hiển thị kết quả
def display_predictions(predictions):
    print("\nKết quả dự đoán lỗi cánh tuabin gió:")
    for model_name, pred in predictions.items():
        print(f"\nMô hình {model_name}:")
        print(f"Loại lỗi dự đoán: {pred['labels'][0]}")
        if pred['probabilities']:
            print("Xác suất cho từng lớp:")
            for label, proba in pred['probabilities'].items():
                print(f"  {label}: {proba:.4f}")

# Lưu các tham số cần thiết từ mã huấn luyện
def save_training_params(scaler, pca, label_encoder):
    joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
    joblib.dump(pca, os.path.join(output_dir, 'pca.pkl'))
    joblib.dump(label_encoder, os.path.join(output_dir, 'label_encoder.pkl'))
    print("Đã lưu các tham số: scaler, pca, label_encoder.")



In [7]:
# Ví dụ sử dụng
if __name__ == "__main__":
    # Ví dụ 1: Nhập danh sách số liệu
    sample_signal = [
    -0.001167, -0.003504, -0.002822, 0.000201, 0.000636, 0.006501, -0.003403, -0.006783, -0.00208,
    -0.000371, 0.001474, -0.001588, 0.004165, 0.000814, 0.000524, -0.00561, 0.001716, 0.003939,
    -0.001839, -0.014185, 0.001023, -0.004787, -0.001749, -0.003032, 0.004793, 0.00584, -0.004466,
    0.005212, -0.001488, 0.003295, 0.000588, 0.003698, -7.82e-05, 0.00236, 0.004229, 0.000814,
    -0.001688, -0.002681, -0.002691, -0.003042, 0.016441, -0.006412, 0.001474, -0.004446, 0.007919,
    0.005727, 0.007419, -0.002832, 0.002022, 0.009417, -0.003454, 0.00054, 0.002151, 0.000975,
    0.005164, 0.001571, -0.003183, -0.004326, 0.000996, -0.007064, 0.001974, -0.003965, -0.00228,
    -0.00227, 0.007113, 0.01077, -0.001378, 0.005599, -0.004126, -0.003263, -0.001979, -0.001237,
    0.002119, -0.003574, 0.004245, 0.004052, -0.007967, -0.001859, 0.011141, -0.00215, 0.002183,
    -0.001368, -0.001719, 0.002634, 0.000298, -0.001678, -0.002822, -0.006693, -0.003393, -0.003995,
    -0.003243, -0.001979, 0.00729, 0.008096, 0.001829, -0.005941, 0.00083, 0.000427, 0.001458,
    -0.001187, 0.000508, 0.001974, 0.000105, -0.00233, -0.002601, -0.004406, -0.002621, 0.012623,
    -0.006232, -0.002661, -0.002701, 0.000801, 0.007403, 0.001587, 0.006533, 0.014363, 0.006759,
    -0.011838, -0.001528, 0.001764, 0.001974, 0.001313, 0.001281, -0.004216, 0.002683, -0.004517,
    -0.01267, -0.007204, -0.003875, -0.003042, -0.00233, -0.001368, 0.0041, 0.010416, 0.002844,
    -0.003273, 0.011109, 0.00236, -0.000176, 0.004197, 0.0012, 0.00112, -0.001999, 0.002538,
    -0.005931, -0.007104, -0.00562, 0.011527, -0.001919, -0.006573, -0.001979, -0.001368, 0.001941,
    0.005695, 0.000749, 0.003859, 0.007661, -0.007395, 0.004052, 0.011527, 0.004535, 0.001265,
    0.004245, -0.005189, 0.004326, -0.001458, -0.001568, -0.001769, -0.004667, 0.000234, -0.005389,
    0.000234, -0.00213, 0.013686, 0.006726, 0.000996, -0.004406, 0.016989, 0.018326, 0.00033,
    -7.82e-05, 0.000943, 0.002296, 0.001506, -0.007706, -0.007596, 0.001168, -0.003243, -0.008689,
    -0.004446, -0.003333, 0.000588, -0.001428, 0.009659, 0.004213, 0.001909, 0.006404, -0.004146,
    -0.005479, 0.001925, 0.003166, 0.002666, 0.001909, -0.002912, -0.002792, 0.005099, -0.001919,
    -0.007746, 0.000312, -0.002411, -0.001057, -0.002491, -0.000273, 0.000491, 0.006275, 0.00584,
    -0.00209, 0.010931, 0.01309, -0.003664, 0.000862, 0.000798, 0.000943, 0.002908, -0.003524,
    -0.002892, -0.004918, 0.002747, -0.008308, -0.001317, 0.002844, -0.001979, 0.000282, 0.002828,
    0.008402, 0.001088, 0.014218, 0.005937, -0.009491, -0.003133, 0.00816, -0.001187, 0.000604,
    0.001587, 0.001845, -0.003283, -0.010725, -0.010664, 0.001039, 0.001136, -0.004767, -0.002701,
    -0.001909, 0.002215, 0.01367, 0.000991, 0.000117, 0.014411, 0.001958, -0.001558, -0.00216,
    0.004696, 0.0017, 0.001281, -0.003674, -0.003544, -0.004697, -0.001628, -0.004657, -0.011958,
    -0.006422, -0.00212, -0.001588, 0.002425, 0.001974, 0.008418, 0.014572, 0.01533, -0.008227,
    -0.006542, 0.012124, -0.001678, 0.002602, -0.000957, 0.00489, -0.003263, 0.004729, -0.013041,
    -0.002992, 0.019051, -0.003925, -0.004115, -0.00233, -0.001899, 0.007774, 0.005792, -0.002531,
    -0.001678, 0.004825, 0.015185, -0.005098, 0.002924, -0.001498, 0.005776, -0.00214, 0.001941,
    -0.001668, -0.003133, 0.003617, -0.011246, -0.006001, -0.003955, -0.001638, -0.00229, 0.006694,
    0.005969, 0.006597, 0.009546, 0.001506, 0.00468, 0.001458, 0.002328, 0.000572, 0.001716,
    -0.003604, -0.003143, -0.00231, -0.002822, 0.005212, 0.01098, 0.003182, -0.003654, -0.002782,
    -0.002661, 0.005969, 0.008418, 0.00112, -0.0022, -0.002932, 0.002521, 0.001007, -0.003062,
    0.000605, 0.002248, 0.004777, -0.003403, -0.00215, -0.010303, 0.001023, -0.009972, -0.012219,
    -0.002611, -0.003133, -0.001678, 0.001184, 0.005711, 0.003085, 0.005776, -0.003805, -0.006723,
    4.03e-05, 0.006114, -0.001368, 0.002151, -0.00205, -0.001889, 0.003279, -0.003885, -0.004166,
    0.002715, 0.005937, -0.00224, -0.004216, -0.0021, 0.000996, 0.012462, -0.002401, 0.002054,
    0.0041, 0.00721, 0.005389, -0.004426, 0.001893, 0.001088, 0.001233, 0.001458, -0.004527,
    -0.003383, 0.003424, -0.009812, -0.015639, -0.0022, -0.003323, 0.001104, -0.003103, 0.005067,
    0.005035, 0.016264, 0.003585, -0.00219, 0.017537, 0.003166, 0.003214, -0.001558, 0.001523,
    0.003166, 0.001313, -0.004136, -0.001207, -0.003383, -0.004136, 0.000169, -0.005871, -0.003383,
    -0.001989, -0.001037, 0.004874, -0.004737, 0.00294, 0.011544, -0.001829, 0.001796, 0.002924,
    0.000137, 0.002795, -0.001959, -0.006091, 0.001055, -0.00226, -0.003062, -0.017594, -0.005008,
    -0.002832, 0.00149, -0.003815, 0.003714, 0.010657, 0.012849, 0.006162, 0.002634, 0.007242,
    0.001877, 0.006098, 0.000218, 0.00286, -0.002862, 0.000701, -0.010203, -0.010755, -0.008298,
    0.01475, 0.001361, -0.007676, -0.000762, -0.003093, 0.001555, 0.003359, 0.00497, 0.004954
]
    try:
        predictions = predict_fault(signal=sample_signal)
        display_predictions(predictions)
    except Exception as e:
        print(f"Lỗi khi dự đoán: {e}")

    # Ví dụ 2: Nhập từ file CSV
    # csv_file = "your_signal.csv"
    # try:
    #     predictions = predict_fault(csv_file=csv_file)
    #     display_predictions(predictions)
    # except Exception as e:
    #     print(f"Lỗi khi dự đoán: {e}")

Lỗi khi dự đoán: Không tìm thấy file mô hình hoặc tham số: [Errno 2] No such file or directory: 'model_outputs/wind_turbine_fault_model_lightgbm.pkl'. Vui lòng chạy mã huấn luyện trước.
